In [162]:
%matplotlib qt
import time
import sys
import math
import numpy as np
import pandas as pd
from datetime import datetime
import calendar
import holidays
import matplotlib.pyplot as plt
import cPickle as pickle
from datetime import datetime
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution1D

targets = None
features = None
with open('../Feature_Selection/targets.pkl') as f:
    targets = np.array(pickle.load(f))
    targets = targets.reshape((targets.shape[0],1))
with open('../Feature_Selection/features.pkl') as f:
    features = np.array(pickle.load(f))

In [181]:
#scale numerical features to (0,1) and one-hot encode categorical features
train_size=0.6
sample_size=targets.shape[0]
target_scaler = preprocessing.MinMaxScaler((0,1))
feature_scaler = preprocessing.MinMaxScaler((0,1))
scaled_targets = target_scaler.fit_transform(targets)
scaled_features = feature_scaler.fit_transform(features)
x = scaled_features
y = scaled_targets
y_train = y[:int(train_size*sample_size),:]
x_train = x[:int(train_size*sample_size),:]

In [182]:
y_val = targets[int(sample_size*train_size):,:]
x_val = x[int(sample_size*train_size):,:]

In [186]:
model = Sequential()
model.add(Dense(30, init = 'glorot_uniform', activation = 'tanh', input_dim = x_train.shape[1]))
model.add(Dense(1, init = 'zero', activation = 'linear'))
model.compile(loss='mean_squared_error', optimizer='sgd', metrics = ['accuracy'])
model.summary()
model.fit(x_train, y_train, nb_epoch=15)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_38 (Dense)                 (None, 30)            2490        dense_input_18[0][0]             
____________________________________________________________________________________________________
dense_39 (Dense)                 (None, 1)             31          dense_38[0][0]                   
Total params: 2521
____________________________________________________________________________________________________
Epoch 1/15
26248/26248 [==============================] - 2s - loss: 7.2977e-04 - acc: 0.0000e+00     
Epoch 2/15
26248/26248 [==============================] - 2s - loss: 5.8004e-04 - acc: 0.0000e+00     
Epoch 3/15
26248/26248 [==============================] - 2s - loss: 4.9015e-04 - acc: 0.0000e+00     
Epoch 4/15
26248/26248 [==============================] - 2s - los

In [187]:
y_pred = model.predict(x_val)
y_pred = target_scaler.inverse_transform(y_pred)
np.mean(np.abs(y_pred-y_val)/y_val)

0.15990945209273635

In [188]:
plt.plot(y_val, label='actual')
plt.plot(y_pred, label='predicted')
plt.legend()
plt.show()

In [159]:
#now just use price-24 and date/time
x2 = np.concatenate((features[:,0].reshape(features.shape[0],1),features[:,-4:]), axis=1)
feature_scaler2 = preprocessing.MinMaxScaler((0,1))
x2 = feature_scaler2.fit_transform(x2)
x_train2 = x2[:int(train_size*sample_size),:]
model = Sequential()
model.add(Dense(2, init = 'glorot_uniform', activation = 'tanh', input_dim = x_train2.shape[1]))
model.add(Dense(1, init = 'zero', activation = 'linear'))
model.compile(loss='mean_squared_error', optimizer='sgd', metrics = ['accuracy'])
model.summary()
model.fit(x_train2, y_train, nb_epoch=15)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_21 (Dense)                 (None, 2)             12          dense_input_11[0][0]             
____________________________________________________________________________________________________
dense_22 (Dense)                 (None, 1)             3           dense_21[0][0]                   
Total params: 15
____________________________________________________________________________________________________
Epoch 1/15
34997/34997 [==============================] - 3s - loss: 6.6194e-04 - acc: 2.8574e-05     
Epoch 2/15
34997/34997 [==============================] - 3s - loss: 6.5461e-04 - acc: 2.8574e-05     
Epoch 3/15
34997/34997 [==============================] - 3s - loss: 6.5260e-04 - acc: 2.8574e-05     
Epoch 4/15
34997/34997 [==============================] - 3s - loss:

In [160]:
x_val2 = x2[int(sample_size*train_size):,:]
y_pred = model.predict(x_val2)
y_pred = target_scaler.inverse_transform(y_pred)
print(np.mean(np.abs(y_pred-y_val)/y_val))
plt.plot(y_val, label='actual')
plt.plot(y_pred, label='predicted')
plt.legend()
plt.show()

0.775147186043


In [165]:
features.shape

(43747, 82)